<a href="https://colab.research.google.com/github/arnaldourbina/desafio_telecom_x/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
data_json = pd.read_json('/content/TelecomX_Data.json')
data_json.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


### Conozce el conjunto de datos

✅ Explorar las columnas del dataset y verificar sus tipos de datos.

✅ Consultar el diccionario para comprender mejor el significado de las variables.

✅ Identificar las columnas más relevantes para el análisis de evasión.

### Diccionario de datos
**customerID:** número de identificación único de cada cliente

**Churn:** si el cliente dejó o no la empresa

**gender:** género (masculino y femenino)

**SeniorCitizen:** información sobre si un cliente tiene o no una edad igual o mayor a 65 años

**Partner:** si el cliente tiene o no una pareja

**Dependents:** si el cliente tiene o no dependientes

**tenure:** meses de contrato del cliente
**PhoneService:** suscripción al servicio telefónico

**MultipleLines:** suscripción a más de una línea telefónica

**InternetService:** suscripción a un proveedor de internet

**OnlineSecurity:** suscripción adicional de seguridad en línea

**OnlineBackup:** suscripción adicional de respaldo en línea

**DeviceProtection:** suscripción adicional de protección del dispositivo

**TechSupport:** suscripción adicional de soporte técnico, menor tiempo de espera

**StreamingTV:** suscripción de televisión por cable

**StreamingMovies:** suscripción de streaming de películas

**Contract:** tipo de contrato

**PaperlessBilling:** si el cliente prefiere recibir la factura en línea

**PaymentMethod:** forma de pago

**Charges.Monthly:** total de todos los servicios del cliente por mes

**Charges.Total:** total gastado por el cliente



In [17]:
data_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


#🔧 Transformación

### Normalizacion de columnas anidadas

In [18]:
columnas_norm = ['customer', 'phone', 'internet', 'account']

data_json_norm = pd.DataFrame()
for columna in columnas_norm:
    data_json_norm = pd.concat([data_json_norm, pd.json_normalize(data_json[columna])], axis=1)
data_json_norm = pd.concat([data_json.drop(columns=columnas_norm), data_json_norm], axis=1)
data_json_norm

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [19]:
data_json_norm.rename(columns={
    'customerID': 'id',
    'Churn':'cancelacion',
    'gender': 'genero',
    'SeniorCitizen': 'mayor_de_65',
    'Partner': 'pareja',
    'Dependents': 'dependientes',
    'tenure': 'meses_de_contrato',
    'PhoneService': 'servicio_telefonico',
    'MultipleLines': 'lineas_multiples',
    'InternetService': 'servicio_internet',
    'OnlineSecurity': 'seguridad_en_linea',
    'OnlineBackup': 'Soporte_en_linea',
    'DeviceProtection': 'protecc_disp',
    'TechSupport': 'soporte_tecnico',
    'StreamingTV': 'servicio_tv',
    'StreamingMovies': 'servicio_peliculas',
    'PaperlessBilling': 'facturas_electronicas',
    'PaymentMethod': 'metodo_pago',
    'Contract': 'tipo_contrato',
    'Charges.Monthly': 'total_pagos_mensuales',
    'Charges.Total': 'ingreso_total_cliente'
}, inplace=True)

In [20]:
data_json_norm.describe()

,mayor_de_65,meses_de_contrato,total_pagos_mensuales
count,7267.000000,7267.000000,7267.000000
mean,0.162653,32.346498,64.720098
std,0.369074,24.571773,30.129572
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.425000
50%,0.000000,29.000000,70.300000
75%,0.000000,55.000000,89.875000
max,1.000000,72.000000,118.750000


In [21]:
#data_json_norm['ingreso_total_cliente'] = data_json_norm['ingreso_total_cliente'].replace('', np.nan)
data_json_norm['ingreso_total_cliente'].apply(lambda x: str(x).strip() == '').sum()

np.int64(11)

In [22]:
data_json_norm['ingreso_total_cliente'] = data_json_norm['ingreso_total_cliente'].apply(
    lambda x: 0 if str(x).strip() == '' else x
)


In [23]:
data_json_norm['ingreso_total_cliente'] = data_json_norm['ingreso_total_cliente'].astype(np.float64)

In [24]:
data_json_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     7267 non-null   object 
 1   cancelacion            7267 non-null   object 
 2   genero                 7267 non-null   object 
 3   mayor_de_65            7267 non-null   int64  
 4   pareja                 7267 non-null   object 
 5   dependientes           7267 non-null   object 
 6   meses_de_contrato      7267 non-null   int64  
 7   servicio_telefonico    7267 non-null   object 
 8   lineas_multiples       7267 non-null   object 
 9   servicio_internet      7267 non-null   object 
 10  seguridad_en_linea     7267 non-null   object 
 11  Soporte_en_linea       7267 non-null   object 
 12  protecc_disp           7267 non-null   object 
 13  soporte_tecnico        7267 non-null   object 
 14  servicio_tv            7267 non-null   object 
 15  serv

### Creación de columna de cuentas diarias

In [25]:
data_json_norm['cuentas_diarias'] = data_json_norm['total_pagos_mensuales'] / 30

### Estandarización y transformación de datos (opcional)

In [26]:
data_json_norm

,id,cancelacion,genero,mayor_de_65,pareja,dependientes,meses_de_contrato,servicio_telefonico,lineas_multiples,servicio_internet,...,protecc_disp,soporte_tecnico,servicio_tv,servicio_peliculas,tipo_contrato,facturas_electronicas,metodo_pago,total_pagos_mensuales,ingreso_total_cliente,cuentas_diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30,2.186667
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40,1.996667
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85,2.463333
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85,3.266667
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40,2.796667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,Yes,No,No,One year,No,Mailed check,55.15,742.90,1.838333
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.70,2.836667
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75,1.676667
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65,2.261667


In [27]:
data_json_norm['tipo_contrato'] = data_json_norm['tipo_contrato'].str.lower()

data_json_norm['metodo_pago'] = data_json_norm['metodo_pago'].str.lower()

data_json_norm['servicio_internet'] = data_json_norm['servicio_internet'].str.lower()

data_json_norm['genero'] = data_json_norm['genero'].str.lower()


In [28]:
columnas_boolean = ['cancelacion', 'pareja', 'dependientes',
                    'servicio_telefonico','seguridad_en_linea',
                    'Soporte_en_linea', 'soporte_tecnico',
                    'servicio_tv', 'servicio_peliculas', 'facturas_electronicas']

In [29]:
data_json_norm[columnas_boolean] = data_json_norm[columnas_boolean].replace({'Yes': 1, 'No': 0})

<ipython-input-29-976709eb2f27>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_json_norm[columnas_boolean] = data_json_norm[columnas_boolean].replace({'Yes': 1, 'No': 0})


In [30]:
data_json_norm['lineas_multiples'] = (data_json_norm['lineas_multiples'].replace({'Yes': 1,'No': 0,
                                                                                  'No phone service': 0}).fillna(0).astype(int))
data_json_norm['seguridad_en_linea'] = (data_json_norm['seguridad_en_linea'].replace({'Yes': 1,'No': 0,
                                                                                  'No internet service': 0}).fillna(0).astype(int))
data_json_norm['Soporte_en_linea'] = (data_json_norm['Soporte_en_linea'].replace({'Yes': 1,'No': 0,
                                                                                  'No internet service': 0}).fillna(0).astype(int))
data_json_norm['protecc_disp'] = (data_json_norm['protecc_disp'].replace({'Yes': 1,'No': 0,
                                                                                  'No internet service': 0}).fillna(0).astype(int))

data_json_norm['soporte_tecnico'] = (data_json_norm['soporte_tecnico'].replace({'Yes': 1,'No': 0,
                                                                                  'No internet service': 0}).fillna(0).astype(int))
data_json_norm['servicio_tv'] = (data_json_norm['servicio_tv'].replace({'Yes': 1,'No': 0,
                                                                                  'No internet service': 0}).fillna(0).astype(int))
data_json_norm['servicio_peliculas'] = (data_json_norm['servicio_peliculas'].replace({'Yes': 1,'No': 0,
                                                                                  'No internet service': 0}).fillna(0).astype(int))

<ipython-input-30-3e6e45c219ff>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_json_norm['lineas_multiples'] = (data_json_norm['lineas_multiples'].replace({'Yes': 1,'No': 0,
<ipython-input-30-3e6e45c219ff>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_json_norm['seguridad_en_linea'] = (data_json_norm['seguridad_en_linea'].replace({'Yes': 1,'No': 0,
<ipython-input-30-3e6e45c219ff>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, exp

In [31]:
#data_json_norm.to_csv('data_telecom_x2.csv', index=False)

#📊 Carga y análisis

In [46]:
data_json_norm['cancelacion'].value_counts()

,count
cancelacion,
0,5174
1,1869


In [47]:
#Eliminar filas vacias
data_json_fil_vacias = data_json_norm[data_json_norm['cancelacion'] == '']
data_json_norm = data_json_norm.drop(data_json_fil_vacias.index)
data_json_norm.reset_index(drop=True, inplace=True)
data_json_norm.head()

,id,cancelacion,genero,mayor_de_65,pareja,dependientes,meses_de_contrato,servicio_telefonico,lineas_multiples,servicio_internet,...,protecc_disp,soporte_tecnico,servicio_tv,servicio_peliculas,tipo_contrato,facturas_electronicas,metodo_pago,total_pagos_mensuales,ingreso_total_cliente,cuentas_diarias
0,0002-ORFBO,0,female,0,1,1,9,1,0,dsl,...,0,1,1,0,one year,1,mailed check,65.6,593.30,2.186667
1,0003-MKNFE,0,male,0,0,0,9,1,1,dsl,...,0,0,0,1,month-to-month,0,mailed check,59.9,542.40,1.996667
2,0004-TLHLJ,1,male,0,0,0,4,1,0,fiber optic,...,1,0,0,0,month-to-month,1,electronic check,73.9,280.85,2.463333
3,0011-IGKFF,1,male,1,1,0,13,1,0,fiber optic,...,1,0,1,1,month-to-month,1,electronic check,98.0,1237.85,3.266667
4,0013-EXCHZ,1,female,1,1,0,3,1,0,fiber optic,...,0,1,1,0,month-to-month,1,mailed check,83.9,267.40,2.796667


In [45]:
data_json_norm['ingreso_total_cliente'].describe()

,ingreso_total_cliente
count,7043.000000
mean,2279.734304
std,2266.794470
min,0.000000
25%,398.550000
50%,1394.550000
75%,3786.600000
max,8684.800000


In [35]:
data_json_norm['total_pagos_mensuales'].describe()

,total_pagos_mensuales
count,7267.000000
mean,64.720098
std,30.129572
min,18.250000
25%,35.425000
50%,70.300000
75%,89.875000
max,118.750000


#📄Informe final